Эволюция автофакера примерно такая:

1. Начинал с языковых моделей, обучил на них N-граммную статистику.
2. Обычный TF-IDF, он показал хороший результат, и сразу стало понятно, что нужно чистить корпус: стоп-слова, опечатки, слипшиеся слова, различный мусор, транслит и так далее. Параллельно смотрел на эмбеддинги и возникла идея умножать tf-idf на вектор слова, но так и не попробовал.
3. KMeans, плюс сразу после него пробовал использовать KMedoids. (https://github.com/letiantian/kmedoids) Там можно ему косинусную метрику дать, потому что kmeans умеет только по евклидову растоянию сравнивать объекты( Ну и ещё: `kmeans - O(n),  а kmedoids - O(n**2)` Результат показался в целом норм, но как будто бы даже хуже, чем просто TF-IDF. В кластеризации непонятно, как в цифрах оценить результат, поэтому оценивали вручную.
4. Дальше на замену KMedoids решил попробовать LSI (svd-разложение), число компонентов брал близко к числу категорий (или числу тем-микротем). Там было что-то около 15-20.
5. Дальше решил посмотреть в сторону лингвистики. Гипотеза была такая: Если провести синтаксический анализ каждого вопроса (для ответов тоже может быть), то для каждого предложения мы получаем действие (сказуемое) на верхнем уровне и подчиняемые сказуемому на нижних уровнях). Так вот можно было бы эти сказуемые/VP разбить на группы (кластеризивать Левенштейном + fasttext’ом/word2vec’ом). А затем для нового вопроса вытащить сказуемое, найти для него группу с помощью w2v+Левенштейна, и для уточнений поиска ответов можно пользоваться, к примеру, вопросительными словами из вопроса (как/когда/где). А затем остаётся NP, которое зависит от VP. Но этот подход оказался сложным в реализации и мы его быстро свернули.
6. Возвращение к первым подходам, так как они показывали хорошие результаты (вектора слов * IDF слова) с заголовком и посмотреть, что получится). У  меня есть заголовок. Надо его засплитить на слова, для каждого слова взять вектор Xi из фасттекста/ворд2века, умножить его на IDF слова Xi*IDFi. Затем взять среднее от всех векторов заголовка, то есть mean(Vi*IDFi, i=0-sentence_length). Получилась какая-то фигня, но возможно я что-то не так делал.
7. Потом пошли эксперименты с классификацией, хотел сделать чтобы автоматически определялась категория вопроса и отсекать вопросы из других категорий, чтобы сделать гибрид потом с каким-нибудь другим подходом. LDA и TruncatedSVD, количество тем пробовал разное, оптимально показалось ~20 тем.
8. Пришли к fasttext’у и остановились на нем пока, он хорошо понимает опечатки (напр., "денюшки" - слово, которого изначально нет в нашем датасете) и дает коэффициенты сравнения, но проблема в том, что его надо дообучить на нашей предметной области.

Текущий результат: все не очень хорошо, без дообучения работает хуже бейзлайна в виде обычного поиска. Но можно дообучать. Веса слов в специфичном корпусе меньше веса слов в большом корпусе (я про спортивные тексты). Сойдет за бейзлайн. Придумали алгоритм получше. Также потестим классификацию, хоть с этим и сложнее